<a href="https://colab.research.google.com/github/esmahoney/Cat-Caption-Cage-Match/blob/main/Copy_of_Cat_Caption_Cage_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cat Caption Cage Match Intro
This program is a game for remote teams. Designed to be used as a quick workshop icebreaker or retro beginner.

Game overview:
1. players join with shared url
2. game facilitator starts game
3. random cat pic is featured
4. players caption the picture
5. AI Gordon Ramsey rates the captions from 1-10
6. Player with highest rating wins the round

** This program is a WIP. additional updates will enable the players to start and complete multiple rounds with scores being stored and used to determine winner over multiple rounds and, later, multiple sessions.

This program is an experimental, learning program that can also be found and reviewed for learning purposed on github. It combines python, AI, remote API integration as well as database creation and updates.

Documenting this project I try to break it down by functionality and phase.

## Gradio Interface and joining capability
This program uses a Gradio Interface for the UI and connects to Gemini API for rating and CAT API to generate the random cat pic (which is optional as it can also be generated - although not as cool - from google).

In [ ]:
import os, getpass, textwrap, io, requests, random, duckdb
from urllib.parse import urlparse

# Gemini / Dev API key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔑  Enter Google GenAI API key: ")

# TheCatAPI key is optional for public endpoints, but safer to add one.
if "THECATAPI_KEY" not in os.environ:
    os.environ["THECATAPI_KEY"] = getpass.getpass("🔑  Enter TheCatAPI key (or hit Enter to skip): ")


In [ ]:
import gradio as gr
from google.generativeai import GenerativeModel
from PIL import Image

model   = GenerativeModel("gemini-2.5-pro")    # text judge
vision  = GenerativeModel("gemini-pro-vision") # multimodal judge

con = duckdb.connect()
con.execute("""
CREATE TABLE IF NOT EXISTS scores (
    round INT,
    player VARCHAR,
    caption VARCHAR,
    score INT
);
""")

# --- Image utilities ---------------------------------------------------------
CAT_API = "https://api.thecatapi.com/v1/images/search?size=med&mime_types=jpg"

def fetch_random_cat() -> Image.Image:
    """Grab a cat picture and return as PIL.Image."""
    headers = {"x-api-key": os.environ.get("THECATAPI_KEY", "")}
    data = requests.get(CAT_API, headers=headers, timeout=10).json()[0]
    url = data["url"]
    img_bytes = requests.get(url, timeout=10).content
    return Image.open(io.BytesIO(img_bytes))

# --- Scoring -----------------------------------------------------------------
judge_prompt = textwrap.dedent(
    """
    You are Cat Meme Gordon Ramsay. Rate the caption below on a scale of 0–10 for humour and relevance to the picture.
    Be ruthless. Reply **only** with the integer score. No additional text.
    """
)

def score_caption(image: Image.Image, caption: str) -> int:
    """Call Gemini to get a numeric score."""
    try:
        resp = vision.generate_content([
            image,
            judge_prompt + "\\nCAPTION: " + caption
        ])
        # Extract first integer 0‑10
        for tok in resp.text.split():
            if tok.isdigit():
                return max(0, min(10, int(tok)))
    except Exception as e:
        print("Gemini error:", e)
    return random.randint(1, 5)  # fallback so game never stalls

In [ ]:
current_round = 0
current_image = None

def new_round():
    global current_round, current_image
    current_round += 1
    current_image = fetch_random_cat()
    return current_round, current_image

round_num, img = new_round()

def submit_caption(player_name, caption):
    score = score_caption(current_image, caption)
    con.execute(
        "INSERT INTO scores VALUES (?, ?, ?, ?)",
        [round_num, player_name.strip() or "Anon", caption.strip(), score]
    )
    df = con.execute("SELECT player, SUM(score) AS total FROM scores GROUP BY player ORDER BY total DESC").df()
    leaderboard_md = "\\n".join([f"**{p}** – {s}" for p, s in df.values]) or "_No scores yet_"
    return f"Gemini gave you **{score}/10**", leaderboard_md

with gr.Blocks(title="Cat-Caption Cage-Match") as demo:
    gr.Markdown("## 🐱 Cat-Caption Cage-Match")
    image_comp = gr.Image(value=img, label="Fresh feline – craft your meme!")
    with gr.Row():
        player_name = gr.Textbox(label="Your name", placeholder="e.g. Alice")
        caption_box = gr.Textbox(label="Caption", placeholder="15 words max…")
    submit_btn = gr.Button("Submit 🏹")
    result = gr.Markdown()
    leaderboard = gr.Markdown("_Scores will appear here_", elem_id="board")
    next_btn = gr.Button("Next Round 🔄")

    submit_btn.click(submit_caption, inputs=[player_name, caption_box], outputs=[result, leaderboard])

    def advance_round():
        rnd, new_img = new_round()
        caption_box.value = ""
        result.value = ""
        return {image_comp: new_img, leaderboard: leaderboard.value}

    next_btn.click(advance_round, outputs=[image_comp, leaderboard])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e684cedded73eeaa7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


```

id: aIHWwsCqkjz2
cell_type: code

In [ ]:
# Check if the 'scores' table exists
tables = con.execute("SHOW TABLES").fetchall()
table_names = [t[0] for t in tables]

if 'scores' in table_names:
    print("The 'scores' table exists.")
else:
    print("The 'scores' table does not exist.")

The 'scores' table exists.


In [ ]:
# Display the contents of the 'scores' table
scores_df = con.execute("SELECT * FROM scores").df()
display(scores_df)

,round,player,caption,score
0,1,Erinn,I own you,3
1,1,Erinn,I will bury you,4
